#### xgb model that combines swing chase value, swing percent, and z-contact%

In [52]:
import pandas as pd
import matplotlib as plt
import numpy as np
import statsmodels.api as sm
import os 
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Swing Decisons')

import data

In [53]:
swing_data = pd.read_csv('./data/batter_data/plate_15_24.csv')
swing_value = pd.read_csv('./data/stat_values/swing_values.csv')

clean names for matching

In [54]:
import unicodedata

def clean_names(df, name_col):
    # Make a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Convert to lowercase and remove leading/trailing whitespace
    df[name_col] = df[name_col].str.lower().str.strip()
    
    # Remove accents
    df[name_col] = df[name_col].apply(lambda x: ''.join(
        c for c in unicodedata.normalize('NFKD', str(x))
        if not unicodedata.combining(c)
    ))
    
    # Keep only letters and replace multiple spaces with single space
    df[name_col] = df[name_col].str.replace('[^a-z ]', '', regex=True).str.replace('\s+', ' ', regex=True)
    
    return df

<>:17: SyntaxWarning: invalid escape sequence '\s'
<>:17: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dalto\AppData\Local\Temp\ipykernel_22984\2103764149.py:17: SyntaxWarning: invalid escape sequence '\s'
  df[name_col] = df[name_col].str.replace('[^a-z ]', '', regex=True).str.replace('\s+', ' ', regex=True)


In [55]:
swing_data = clean_names(swing_data, 'Name')
swing_value = clean_names(swing_value, 'batter')

In [56]:
df = swing_data.merge(swing_value[['game_date','batter','chase_shadow_value', 'swing_value']], left_on=['Season','Name'], right_on=['game_date', 'batter'], how='left')
df = df.drop(columns=['Team', 'NameASCII', 'PlayerId', 'MLBAMID', 
                       'K%', 'O-Swing%', 'BB%'])
df = df[df['chase_shadow_value'].notna()]
df = df[['batter', 'PA', 'game_date','chase_shadow_value', 'swing_value', 'Swing%']]

In [57]:
Y_Chase = df['chase_shadow_value']
X_swing = df['Swing%']
X_swing = sm.add_constant(X_swing) # Add intercept

model_for_residuals = sm.OLS(Y_Chase, X_swing)
results_for_residuals = model_for_residuals.fit()

        
print(results_for_residuals.summary())

df['adj_chase_shadow_value'] = results_for_residuals.resid
# You can verify that 'AdjustedSwingDecisionStat' has low correlation with 'SwingPercent'
correlation = df['adj_chase_shadow_value'].corr(df['Swing%'])
print(f"\nCorrelation {correlation:.4f}")

                            OLS Regression Results                            
Dep. Variable:     chase_shadow_value   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.707
Method:                 Least Squares   F-statistic:                     5595.
Date:                Mon, 12 May 2025   Prob (F-statistic):               0.00
Time:                        09:23:38   Log-Likelihood:                -6471.5
No. Observations:                2323   AIC:                         1.295e+04
Df Residuals:                    2321   BIC:                         1.296e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        155.0593      0.741    209.176      0.0

In [58]:
Y_Chase = df['swing_value']
X_swing = df['Swing%']
X_swing = sm.add_constant(X_swing) # Add intercept

model_for_residuals = sm.OLS(Y_Chase, X_swing)
results_for_residuals = model_for_residuals.fit()

        
print(results_for_residuals.summary())

df['adj_swing_value'] = results_for_residuals.resid 
correlation = df['adj_swing_value'].corr(df['Swing%'])
print(f"\nCorrelation {correlation:.4f}")

                            OLS Regression Results                            
Dep. Variable:            swing_value   R-squared:                       0.624
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     3853.
Date:                Mon, 12 May 2025   Prob (F-statistic):               0.00
Time:                        09:23:38   Log-Likelihood:                -9357.0
No. Observations:                2323   AIC:                         1.872e+04
Df Residuals:                    2321   BIC:                         1.873e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        258.2783      2.567    100.613      0.0

In [62]:
df = df[df['PA'] > 100]

In [ ]:
df.to_csv('./data/stat_values/swing_adj_data.csv')